In [ ]:
# !pip install -q -U bitsandbytes
# !pip install -q -U datasets
# !pip install -q -U git+https://github.com/huggingface/transformers.git
# !pip install -q -U git+https://github.com/huggingface/peft.git
# !pip install -q -U git+https://github.com/huggingface/accelerate.git
# !pip install -q -U loralib
# !pip install -q -U einops

In [ ]:
import os
import torch
import numpy as np
from sklearn.metrics import accuracy_score
from datasets import load_dataset

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    TrainerCallback,
    DataCollatorWithPadding,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig,
    set_seed
)

from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    PeftModel,
    PeftConfig
)

In [ ]:
set_seed(42)
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

## **Load & Preprocess Dataset**

In [ ]:
dataset = load_dataset("uit-nlp/vietnamese_students_feedback", trust_remote_code=True)

In [ ]:
print("Dataset structure:", dataset)

Dataset structure: DatasetDict({
    train: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['sentence', 'sentiment', 'topic'],
        num_rows: 3166
    })
})


In [ ]:
def preprocess(example):
    return {
        "text": str(example["sentence"]),
        "label": int(example["sentiment"])
    }

dataset = dataset.map(
    preprocess,
    remove_columns=["sentence", "sentiment", "topic"]
)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3166
    })
})

## **Load Pre-trained Model**

In [ ]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    num_labels=3,
    quantization_config=bnb_config,
    id2label={0: "tiêu_cực", 1: "trung_lập", 2: "tích_cực"},
    label2id={"tiêu_cực": 0, "trung_lập": 1, "tích_cực": 2},
    pad_token_id=tokenizer.pad_token_id
)

model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="SEQ_CLS",
    inference_mode=False
)
model = get_peft_model(model, peft_config)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.print_trainable_parameters()

trainable params: 1,132,544 || all params: 1,035,651,072 || trainable%: 0.1094


In [ ]:
def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=256,
        padding="max_length",
        add_special_tokens=True
    )
    return {
        "input_ids": tokenized["input_ids"],
        "attention_mask": tokenized["attention_mask"],
        "labels": examples["label"]
    }

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text", "label"]
)

Map:   0%|          | 0/1583 [00:00<?, ? examples/s]

Map: 100%|██████████| 1583/1583 [00:00<00:00, 8710.73 examples/s]


In [ ]:
print("\nDataset sau khi xử lý:")
print(tokenized_dataset)

print("\nVí dụ 1 mẫu train:")
print(tokenized_dataset["train"][0])


Dataset sau khi xử lý:
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 11426
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1583
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3166
    })
})

Ví dụ 1 mẫu train:
{'input_ids': [1, 20343, 4005, 29976, 29877, 534, 30097, 29876, 29882, 29871, 30128, 30884, 29891, 29871, 30128, 31556, 869, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

### **Test pre-trained model**

In [ ]:
def test_output_before_training(model, tokenizer, samples=None):
    if samples is None:
        samples = [
            "Giảng viên giảng bài dễ hiểu và nhiệt tình.",
            "Em cảm thấy rất áp lực với lịch học dày đặc.",
            "Bình thường, không có gì đặc biệt lắm."
        ]

    model.eval()
    print("==> Test kết quả dự đoán (model chưa được fine-tuned):\n")
    for i, text in enumerate(samples, 1):
        inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256, padding=True).to(model.device)
        with torch.no_grad():
            outputs = model(**inputs)
            pred_class = torch.argmax(outputs.logits, dim=-1).item()
            pred_label = model.config.id2label[pred_class]
        print(f"{i}. \"{text}\"\n→ Dự đoán: {pred_label} (label id: {pred_class})\n")


In [ ]:
test_output_before_training(model, tokenizer)

==> Test kết quả dự đoán (model chưa được fine-tuned):

1. "Giảng viên giảng bài dễ hiểu và nhiệt tình."
→ Dự đoán: tiêu_cực (label id: 0)

2. "Em cảm thấy rất áp lực với lịch học dày đặc."
→ Dự đoán: tích_cực (label id: 2)

3. "Bình thường, không có gì đặc biệt lắm."
→ Dự đoán: tiêu_cực (label id: 0)



## **Training**

### **Config & Train**

In [ ]:
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    pad_to_multiple_of=8,
    return_tensors="pt"
)

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy_score(labels, preds)}

In [ ]:
class LossLoggerCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs and "loss" in logs:
            print(f"[Step {state.global_step:>5}] Loss: {logs['loss']:.4f}")

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-5,
    weight_decay=0.01,
    fp16=True,
    gradient_checkpointing=True,
    remove_unused_columns=True,
    logging_dir="./logs",
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    optim="paged_adamw_8bit",
    label_names=["labels"]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    callbacks=[LossLoggerCallback()]
)

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Epoch,Training Loss,Validation Loss,Accuracy
0,0.315300,0.361517,0.889450


[Step    50] Loss: 0.8701
[Step   100] Loss: 0.8140
[Step   150] Loss: 0.7435
[Step   200] Loss: 0.6627
[Step   250] Loss: 0.5857
[Step   300] Loss: 0.5121
[Step   350] Loss: 0.4980
[Step   400] Loss: 0.3722
[Step   450] Loss: 0.3914
[Step   500] Loss: 0.3334
[Step   550] Loss: 0.4315
[Step   600] Loss: 0.3760
[Step   650] Loss: 0.3143
[Step   700] Loss: 0.4671
[Step   750] Loss: 0.3661
[Step   800] Loss: 0.3844
[Step   850] Loss: 0.3439
[Step   900] Loss: 0.3587
[Step   950] Loss: 0.3642
[Step  1000] Loss: 0.3569
[Step  1050] Loss: 0.4159
[Step  1100] Loss: 0.3499
[Step  1150] Loss: 0.3674
[Step  1200] Loss: 0.4149
[Step  1250] Loss: 0.4712
[Step  1300] Loss: 0.2821
[Step  1350] Loss: 0.3473
[Step  1400] Loss: 0.3153


TrainOutput(global_step=1428, training_loss=0.4428504211228101, metrics={'train_runtime': 1212.3976, 'train_samples_per_second': 9.424, 'train_steps_per_second': 1.178, 'total_flos': 1.7022865278763008e+16, 'train_loss': 0.4428504211228101, 'epoch': 0.9998249606161387})

In [ ]:
# torch.cuda.empty_cache()

### **Evaluate on Test**

In [ ]:
test_dataset = tokenized_dataset["test"]
print("→ Test set:", len(test_dataset), "samples")

→ Test set: 3166 samples


In [ ]:
test_results = trainer.evaluate(eval_dataset=test_dataset)
print("\nEvaluate on test set:")
print({k: round(v, 4) for k, v in test_results.items()})


Evaluate on test set:
{'eval_loss': 0.4127, 'eval_accuracy': 0.879, 'eval_runtime': 95.8017, 'eval_samples_per_second': 33.047, 'eval_steps_per_second': 16.524, 'epoch': 0.9998}


In [ ]:
test_samples = [
    "Giảng viên giảng bài rất dễ hiểu, em cảm thấy hứng thú với môn học.",
    "Lịch học quá dày, em cảm thấy rất mệt mỏi và không tiếp thu được bài.",
    "Em thấy môn học này khá ổn, không có gì đặc biệt.",
    "Thầy cô tận tình, tuy nhiên hệ thống lớp học trực tuyến hay bị lỗi.",
    "Môn này khó quá, em học mãi không hiểu gì hết.",
    "Quá tuyệt vời, không thể nào tốt hơn"
]

print("\nKết quả phân loại cảm xúc:")
model.eval()
for i, text in enumerate(test_samples, 1):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=256).to(model.device)
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits).item()
    predicted_label = model.config.id2label[predicted_class]
    print(f"{i}. {text}\n→ {predicted_label}\n")


Kết quả phân loại cảm xúc:
1. Giảng viên giảng bài rất dễ hiểu, em cảm thấy hứng thú với môn học.
→ tích_cực

2. Lịch học quá dày, em cảm thấy rất mệt mỏi và không tiếp thu được bài.
→ tiêu_cực

3. Em thấy môn học này khá ổn, không có gì đặc biệt.
→ tiêu_cực

4. Thầy cô tận tình, tuy nhiên hệ thống lớp học trực tuyến hay bị lỗi.
→ tiêu_cực

5. Môn này khó quá, em học mãi không hiểu gì hết.
→ tiêu_cực

6. Quá tuyệt vời, không thể nào tốt hơn
→ tích_cực



In [ ]:
model_save_path = "./fine_tuned_model"
model.save_pretrained(model_save_path, safe_serialization=True)
tokenizer.save_pretrained(model_save_path)

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/chat_template.jinja',
 './fine_tuned_model/tokenizer.json')

## **Inference**

In [ ]:
model_save_path = "./fine_tuned_model"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

base_model = AutoModelForSequenceClassification.from_pretrained(
    "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    quantization_config=bnb_config,
    num_labels=3,
    id2label={0: "tiêu_cực", 1: "trung_lập", 2: "tích_cực"},
    device_map="auto"
)

model = PeftModel.from_pretrained(base_model, model_save_path)
model = model.merge_and_unload()

tokenizer = AutoTokenizer.from_pretrained(model_save_path)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at TinyLlama/TinyLlama-1.1B-Chat-v1.0 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def predict_sentiment(text, model, tokenizer):
    inputs = tokenizer(
        text,
        truncation=True,
        max_length=256,
        padding=True,
        return_tensors="pt"
    ).to(model.device)

    model.eval()
    with torch.no_grad():
        outputs = model(**inputs)

    probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)
    predicted_class = torch.argmax(probabilities).item()
    predicted_label = model.config.id2label[predicted_class]
    confidence = probabilities[0][predicted_class].item()

    return {
        "text": text,
        "label": predicted_label,
        "confidence": round(confidence, 4),
        "probabilities": {
            "tiêu_cực": round(probabilities[0][0].item(), 4),
            "trung_lập": round(probabilities[0][1].item(), 4),
            "tích_cực": round(probabilities[0][2].item(), 4)
        }
    }

In [ ]:
test_samples = [
    "Giáo viên dạy rất nhiệt tình và dễ hiểu",
    "Phòng học quá chật hẹp và thiếu trang thiết bị",
    "Môn học này khá thú vị nhưng lượng bài tập hơi nhiều",
    "Em thấy bình thường",
    "Giảng viên nói về môn học này, nghe như đang kể chuyện cổ tích, khó tin nhưng thú vị.",
    "Cơ sở vật chất tuyệt vời, chỉ thiếu mỗi việc có thể đưa tôi lên mặt trăng.",
    "Môn học này, tôi chắc chắn là mình đã học, nhưng liệu có thể nhớ được không thì chưa biết."
]


for sample in test_samples:
    result = predict_sentiment(sample, model, tokenizer)
    print(f"→ Văn bản: {result['text']}")
    print(f"   Nhãn dự đoán: {result['label']} (Độ tin cậy: {result['confidence']*100:.2f}%)")
    print(f"   Phân phối xác suất: {result['probabilities']}\n")

→ Văn bản: Giáo viên dạy rất nhiệt tình và dễ hiểu
   Nhãn dự đoán: tích_cực (Độ tin cậy: 90.82%)
   Phân phối xác suất: {'tiêu_cực': 0.0535, 'trung_lập': 0.0381, 'tích_cực': 0.9082}

→ Văn bản: Phòng học quá chật hẹp và thiếu trang thiết bị
   Nhãn dự đoán: tiêu_cực (Độ tin cậy: 85.21%)
   Phân phối xác suất: {'tiêu_cực': 0.8521, 'trung_lập': 0.005, 'tích_cực': 0.1431}

→ Văn bản: Môn học này khá thú vị nhưng lượng bài tập hơi nhiều
   Nhãn dự đoán: tích_cực (Độ tin cậy: 83.11%)
   Phân phối xác suất: {'tiêu_cực': 0.1355, 'trung_lập': 0.0334, 'tích_cực': 0.8311}

→ Văn bản: Em thấy bình thường
   Nhãn dự đoán: tích_cực (Độ tin cậy: 59.33%)
   Phân phối xác suất: {'tiêu_cực': 0.2177, 'trung_lập': 0.1891, 'tích_cực': 0.5933}

→ Văn bản: Giảng viên nói về môn học này, nghe như đang kể chuyện cổ tích, khó tin nhưng thú vị.
   Nhãn dự đoán: tiêu_cực (Độ tin cậy: 66.36%)
   Phân phối xác suất: {'tiêu_cực': 0.6636, 'trung_lập': 0.0477, 'tích_cực': 0.2891}

→ Văn bản: Cơ sở vật chất tuyệt vời